In [ ]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

In [ ]:
df[df.course == 'data-engineering-zoomcamp'].head()

In [ ]:
docs_example = [
    "January course details, register now",
    "Course prerequisites listed in January catalog",
    "Submit January course homework by end of month",
    "Register for January course, no prerequisites",
    "January course setup: Python and Google Cloud"
]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
cv.fit(docs_example)

In [ ]:
names = cv.get_feature_names_out()
names

In [ ]:
X = cv.transform(docs_example)

In [ ]:
X.toarray()

In [ ]:
df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

In [ ]:
cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
cv = TfidfVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)

In [ ]:
query = "Do I need to know python to sign up for the January course?"

In [ ]:
q = cv.transform([query])
q.toarray()

In [ ]:
query_dict = dict(zip(names, q.toarray()[0]))
query_dict

In [ ]:
doc_dict = dict(zip(names, X.toarray()[1]))
doc_dict

In [ ]:
df_qd = pd.DataFrame([query_dict, doc_dict], index=['query', 'doc']).T

In [ ]:
(df_qd['query'] * df_qd['doc']).sum()

In [ ]:
X.dot(q.T).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(X, q)

In [ ]:
df.columns

In [ ]:
fields = ['section', 'question', 'text']
transformers = {}
matrices = {}

for field in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=3)
    X = cv.fit_transform(df[field])

    transformers[field] = cv
    matrices[field] = X

In [ ]:
transformers['text'].get_feature_names_out()

In [ ]:
matrices['text']

In [ ]:
query = "I just singned up. Is it too late to join the course?"

In [ ]:
q = transformers['text'].transform([query])
score = cosine_similarity(matrices['text'], q).flatten()

In [ ]:
mask = (df.course == 'data-engineering-zoomcamp').values
score = score * mask
score[:10]

In [ ]:
import numpy as np

In [ ]:
idx = np.argsort(-score)[:10]
idx

In [ ]:
score[idx]

In [ ]:
df.iloc[idx].text

In [ ]:
fields

In [ ]:
query = "I just signed up. Is it too late to join the course?"